In [22]:
import numpy as np
import os
import pandas as pd
import pickle
import xgboost as xgb
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [29]:
df = pd.read_csv('data/processed_files/df_datos_completos.csv')

In [30]:
# Crear una lista de columnas para eliminar que contengan 'titu-' o 'les-'
columns_to_drop = df.filter(like='titu-').columns.tolist() + df.filter(like='les-').columns.tolist()

# Añadir las columnas adicionales a la lista de columnas a eliminar
columns_to_drop += ['index', 'fixture_id', 'resultado', 'goles_local', 'goles_totales', 'goles_visitante', 'goles_descanso_local', 'goles_descanso_visitante', 'fecha_timestamp']

# Eliminar las columnas en la lista columns_to_drop de DataFrame df_partidos
X = df.drop(columns_to_drop, axis=1)
y = df['goles_totales']

In [31]:
X.head()

,id_equipo_local,id_equipo_visitante,arbitro,estadio,season,shots_on_goal_local,shots_on_goal_away,shots_off_goal_local,shots_off_goal_away,total_shots_local,...,goalkeeper_saves_away,total_pass_local,total_pass_away,odd_1,odd_x,odd_2,odd_mas_25,odd_menos_25,tiros_para_marcar_local,tiros_para_marcar_away
0,530,539,"Alfonso Alvarez Izquierdo, Spain",Estadio Vicente Calderón (Madrid),2014,5.0,3.0,5.0,4.0,0.0,...,3.0,0.0,0.0,1.20,6.5,17.00,1.75,2.28,9.25,11.333333
1,536,538,"Carlos Velasco Carballo, Spain",Estadio Ramón Sánchez Pizjuán (Sevilla),2014,6.0,1.0,3.0,7.0,0.0,...,4.0,0.0,0.0,1.67,3.8,5.50,1.81,2.19,9.00,7.500000
2,797,533,"Carlos Clos Gomez, Spain",Estadio Manuel Martínez Valero (Elche),2014,7.0,3.0,5.0,6.0,0.0,...,5.0,0.0,0.0,5.00,3.6,1.75,2.30,1.77,25.00,12.000000
3,544,531,"Alberto Undiano, Spain",Estadio Municipal de Riazor (A Coruña (La Coru...,2014,7.0,5.0,4.0,8.0,0.0,...,6.0,0.0,0.0,3.20,3.1,2.40,2.56,1.60,44.00,11.333333
4,535,723,"Fernando Teixeira Vitienes, Spain",Estadio La Rosaleda (Málaga),2014,8.0,5.0,2.0,2.0,0.0,...,6.0,0.0,0.0,1.53,4.1,6.50,2.04,1.99,6.00,11.000000


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Pipeline para codificar la columna 'arbitro' con OneHotEncoder
arbitro_pipeline = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# Pipeline para codificar la columna 'estadio' con TargetEncoder
estadio_pipeline = Pipeline([
    ('target', TargetEncoder())
])

# ColumnTransformer para aplicar los pipelines a las columnas correspondientes
preprocessor = ColumnTransformer([
    ('arbitro', arbitro_pipeline, ['arbitro']),
    ('estadio', estadio_pipeline, ['estadio']),
    ], remainder = "passthrough")

# Pipeline final con el preprocesamiento y el modelo RandomForestClassifier
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()), #En este caso el escalado de variables funciona mucho peor
    ('xgbr', xgb.XGBRegressor())
])

param_grid = {
    'xgbr__n_estimators': [50, 100, 200, 300, 400, 500],
    'xgbr__learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2, 0.3],
    'xgbr__max_depth': [3, 4, 5, 6, 7, 8],
    'xgbr__min_child_weight': [1, 2, 3, 4, 5, 6],
    'xgbr__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'xgbr__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'xgbr__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 5],
    'xgbr__reg_alpha': [0, 0.1, 0.5, 1, 2, 5],
    'xgbr__reg_lambda': [0.1, 0.5, 1, 2, 5, 10]
}


In [33]:
random_search = RandomizedSearchCV(
    estimator=pipeline_xgb,
    param_distributions=param_grid,
    n_iter=200,  # Número de combinaciones aleatorias de hiperparámetros a probar
    scoring='neg_mean_squared_error',
    cv=3,  # Número de divisiones de validación cruzada
    verbose=2,
    n_jobs=-1,  # Utilizar todos los núcleos disponibles
    random_state=42)

In [34]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('arbitro',
                                                                               Pipeline(steps=[('onehot',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse=False))]),
                                                                               ['arbitro']),
                                                                              ('estadio',
                                                                               Pipeline(steps=[('target',
                                                                                                TargetEncoder())]),
                                                                               ['estadio'])])),
                                             ('scaler', StandardScaler()),
                                             ('xgbr',
                                              XGBRegressor(ba...
                                                        0.5, 1, 2, 5],
                                        'xgbr__learning_rate': [0.01, 0.03,
                                                                0.05, 0.1, 0.2,
                                                                0.3],
                                        'xgbr__max_depth': [3, 4, 5, 6, 7, 8],
                                        'xgbr__min_child_weight': [1, 2, 3, 4,
                                                                   5, 6],
                                        'xgbr__n_estimators': [50, 100, 200,
                                                               300, 400, 500],
                                        'xgbr__reg_alpha': [0, 0.1, 0.5, 1, 2,
                                                            5],
                                        'xgbr__reg_lambda': [0.1, 0.5, 1, 2, 5,
                                                             10],
                                        'xgbr__subsample': [0.5, 0.6, 0.7, 0.8,
                                                            0.9, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [35]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predecir valores en el conjunto de prueba
y_pred = random_search.predict(X_test)

# Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

MSE: 0.25459778866445165
RMSE: 0.5045768411891807
MAE: 0.3353836412357721
R²: 0.8964541005103391


In [36]:
df_corners = pd.read_csv('data/processed_files/df_datos_completos.csv')

In [37]:
df_corners['corners_total'] = df_corners['corners_local'] + df_corners['corners_away']

In [38]:
# Crear una lista de columnas para eliminar que contengan 'titu-' o 'les-'
columns_to_drop = df_corners.filter(like='titu-').columns.tolist() + df_corners.filter(like='les-').columns.tolist()

# Añadir las columnas adicionales a la lista de columnas a eliminar
columns_to_drop += ['index', 'fixture_id', 'resultado', 'goles_local', 'goles_totales', 'goles_visitante', 'goles_descanso_local', 'goles_descanso_visitante', 'fecha_timestamp','corners_total','corners_local','corners_away']

# Eliminar las columnas en la lista columns_to_drop de DataFrame df_partidos
X = df_corners.drop(columns_to_drop, axis=1)
y = df_corners['corners_total']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Pipeline para codificar la columna 'arbitro' con OneHotEncoder
arbitro_pipeline = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# Pipeline para codificar la columna 'estadio' con TargetEncoder
estadio_pipeline = Pipeline([
    ('target', TargetEncoder())
])

# ColumnTransformer para aplicar los pipelines a las columnas correspondientes
preprocessor = ColumnTransformer([
    ('arbitro', arbitro_pipeline, ['arbitro']),
    ('estadio', estadio_pipeline, ['estadio']),
    ], remainder = "passthrough")

# Pipeline final con el preprocesamiento y el modelo RandomForestClassifier
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()), #En este caso el escalado de variables funciona mucho peor
    ('xgbr', xgb.XGBRegressor())
])

param_grid = {
    'xgbr__n_estimators': [50, 100, 200, 300, 400, 500],
    'xgbr__learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2, 0.3],
    'xgbr__max_depth': [3, 4, 5, 6, 7, 8],
    'xgbr__min_child_weight': [1, 2, 3, 4, 5, 6],
    'xgbr__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'xgbr__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'xgbr__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 5],
    'xgbr__reg_alpha': [0, 0.1, 0.5, 1, 2, 5],
    'xgbr__reg_lambda': [0.1, 0.5, 1, 2, 5, 10]
}


In [40]:
random_search_corner = RandomizedSearchCV(
    estimator=pipeline_xgb,
    param_distributions=param_grid,
    n_iter=200,  # Número de combinaciones aleatorias de hiperparámetros a probar
    scoring='neg_mean_squared_error',
    cv=3,  # Número de divisiones de validación cruzada
    verbose=2,
    n_jobs=-1,  # Utilizar todos los núcleos disponibles
    random_state=42)

In [42]:
random_search_corner.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('arbitro',
                                                                               Pipeline(steps=[('onehot',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse=False))]),
                                                                               ['arbitro']),
                                                                              ('estadio',
                                                                               Pipeline(steps=[('target',
                                                                                                TargetEncoder())]),
                                                                               ['estadio'])])),
                                             ('scaler', StandardScaler()),
                                             ('xgbr',
                                              XGBRegressor(ba...
                                                        0.5, 1, 2, 5],
                                        'xgbr__learning_rate': [0.01, 0.03,
                                                                0.05, 0.1, 0.2,
                                                                0.3],
                                        'xgbr__max_depth': [3, 4, 5, 6, 7, 8],
                                        'xgbr__min_child_weight': [1, 2, 3, 4,
                                                                   5, 6],
                                        'xgbr__n_estimators': [50, 100, 200,
                                                               300, 400, 500],
                                        'xgbr__reg_alpha': [0, 0.1, 0.5, 1, 2,
                                                            5],
                                        'xgbr__reg_lambda': [0.1, 0.5, 1, 2, 5,
                                                             10],
                                        'xgbr__subsample': [0.5, 0.6, 0.7, 0.8,
                                                            0.9, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predecir valores en el conjunto de prueba
y_pred = random_search_corner.predict(X_test)

# Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

MSE: 8.470175959640509
RMSE: 2.9103566722380454
MAE: 2.320356240418357
R²: 0.17116905680562544


In [44]:
df_tarjetas = pd.read_csv('data/processed_files/df_datos_completos.csv')

In [45]:
df_tarjetas['tarjetas_total'] = df_tarjetas['yellow_cards_local'] + df_tarjetas['yellow_cards_away'] + df_tarjetas['red_cards_local'] + df_tarjetas['red_cards_away']

In [46]:
# Crear una lista de columnas para eliminar que contengan 'titu-' o 'les-'
columns_to_drop = df_tarjetas.filter(like='titu-').columns.tolist() + df_tarjetas.filter(like='les-').columns.tolist()

# Añadir las columnas adicionales a la lista de columnas a eliminar
columns_to_drop += ['index', 'fixture_id', 'resultado', 'goles_local', 'goles_totales', 'goles_visitante', 'goles_descanso_local', 'goles_descanso_visitante', 'fecha_timestamp','tarjetas_total','yellow_cards_local','yellow_cards_away','red_cards_local','red_cards_away']

# Eliminar las columnas en la lista columns_to_drop de DataFrame df_partidos
X = df_tarjetas.drop(columns_to_drop, axis=1)
y = df_tarjetas['tarjetas_total']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Pipeline para codificar la columna 'arbitro' con OneHotEncoder
arbitro_pipeline = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# Pipeline para codificar la columna 'estadio' con TargetEncoder
estadio_pipeline = Pipeline([
    ('target', TargetEncoder())
])

# ColumnTransformer para aplicar los pipelines a las columnas correspondientes
preprocessor = ColumnTransformer([
    ('arbitro', arbitro_pipeline, ['arbitro']),
    ('estadio', estadio_pipeline, ['estadio']),
    ], remainder = "passthrough")

# Pipeline final con el preprocesamiento y el modelo RandomForestClassifier
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()), #En este caso el escalado de variables funciona mucho peor
    ('xgbr', xgb.XGBRegressor())
])

param_grid = {
    'xgbr__n_estimators': [50, 100, 200, 300, 400, 500],
    'xgbr__learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2, 0.3],
    'xgbr__max_depth': [3, 4, 5, 6, 7, 8],
    'xgbr__min_child_weight': [1, 2, 3, 4, 5, 6],
    'xgbr__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'xgbr__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'xgbr__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 5],
    'xgbr__reg_alpha': [0, 0.1, 0.5, 1, 2, 5],
    'xgbr__reg_lambda': [0.1, 0.5, 1, 2, 5, 10]
}


In [48]:
random_search_tarjetas = RandomizedSearchCV(
    estimator=pipeline_xgb,
    param_distributions=param_grid,
    n_iter=200,  # Número de combinaciones aleatorias de hiperparámetros a probar
    scoring='neg_mean_squared_error',
    cv=3,  # Número de divisiones de validación cruzada
    verbose=2,
    n_jobs=-1,  # Utilizar todos los núcleos disponibles
    random_state=42)

In [49]:
random_search_tarjetas.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('arbitro',
                                                                               Pipeline(steps=[('onehot',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse=False))]),
                                                                               ['arbitro']),
                                                                              ('estadio',
                                                                               Pipeline(steps=[('target',
                                                                                                TargetEncoder())]),
                                                                               ['estadio'])])),
                                             ('scaler', StandardScaler()),
                                             ('xgbr',
                                              XGBRegressor(ba...
                                                        0.5, 1, 2, 5],
                                        'xgbr__learning_rate': [0.01, 0.03,
                                                                0.05, 0.1, 0.2,
                                                                0.3],
                                        'xgbr__max_depth': [3, 4, 5, 6, 7, 8],
                                        'xgbr__min_child_weight': [1, 2, 3, 4,
                                                                   5, 6],
                                        'xgbr__n_estimators': [50, 100, 200,
                                                               300, 400, 500],
                                        'xgbr__reg_alpha': [0, 0.1, 0.5, 1, 2,
                                                            5],
                                        'xgbr__reg_lambda': [0.1, 0.5, 1, 2, 5,
                                                             10],
                                        'xgbr__subsample': [0.5, 0.6, 0.7, 0.8,
                                                            0.9, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [51]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predecir valores en el conjunto de prueba
y_pred = random_search_tarjetas.predict(X_test)

# Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

MSE: 4.694884746350783
RMSE: 2.1667682724165
MAE: 1.7269184335279066
R²: 0.09175968519918032
